source: https://huggingface.co/THUDM/cogvlm-chat-hf 

In [1]:
pip install torch==2.1.0 transformers==4.35.0 accelerate==0.24.1 sentencepiece==0.1.99 einops==0.7.0 xformers==0.0.22.post7 triton==2.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 3.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 9.8 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement triton==2.1.0 (from versions: none)
ERROR: No matching distribution found for triton==2.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xformers==0.0.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 7.9 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 MB 7.5 MB/s eta 0:00:0000:0100:01
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [290 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-310
      creating build/lib.macosx-10.9-x86_64-cpython-310/xformers
      copying xformers/_deprecation_warning.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers
      copying xformers/version.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers
      copying xformers/checkpoint.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers
      copying xformers/__init__.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers
      copying xformers/test.py -> build/lib.macosx-10.9-x86

      copying xformers/benchmarks/LRA/run_tasks.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers/benchmarks/LRA
      copying xformers/benchmarks/LRA/run_grid_search.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers/benchmarks/LRA
      creating build/lib.macosx-10.9-x86_64-cpython-310/xformers/benchmarks/LRA/code
      copying xformers/benchmarks/LRA/code/__init__.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers/benchmarks/LRA/code
      copying xformers/benchmarks/LRA/code/model_wrapper.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers/benchmarks/LRA/code
      copying xformers/benchmarks/LRA/code/dataset.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers/benchmarks/LRA/code
      creating build/lib.macosx-10.9-x86_64-cpython-310/xformers/ops/triton
      copying xformers/ops/triton/__init__.py -> build/lib.macosx-10.9-x86_64-cpython-310/xformers/ops/triton
      copying xformers/ops/triton/rope_padded_kernels.py -> build/lib.macosx-10.9-x86_64-cpython-

Failed to build xformers
ERROR: Could not build wheels for xformers, which is required to install pyproject.toml-based projects
Note: you may need to restart the kernel to use updated packages.


In [1]:
import xformers

ModuleNotFoundError: No module named 'xformers'

In [3]:
import torch
import requests
from PIL import Image
from transformers import AutoModelForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')
model = AutoModelForCausalLM.from_pretrained(
    'THUDM/cogvlm-chat-hf',
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to('cuda').eval()


# chat example
query = 'Describe this image'
image = Image.open(requests.get('https://github.com/THUDM/CogVLM/blob/main/examples/1.png?raw=true', stream=True).raw).convert('RGB')
inputs = model.build_conversation_input_ids(tokenizer, query=query, history=[], images=[image])  # chat mode
inputs = {
    'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
    'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
    'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
    'images': [[inputs['images'][0].to('cuda').to(torch.bfloat16)]],
}
gen_kwargs = {"max_length": 2048, "do_sample": False}

with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0]))

# This image captures a moment from a basketball game. Two players are prominently featured: one wearing a yellow jersey with the number
# 24 and the word 'Lakers' written on it, and the other wearing a navy blue jersey with the word 'Washington' and the number 34. The player
# in yellow is holding a basketball and appears to be dribbling it, while the player in navy blue is reaching out with his arm, possibly
# trying to block or defend. The background shows a filled stadium with spectators, indicating that this is a professional game.</s>



# vqa example
query = 'How many houses are there in this cartoon?'
image = Image.open(requests.get('https://github.com/THUDM/CogVLM/blob/main/examples/3.jpg?raw=true', stream=True).raw).convert('RGB')
inputs = model.build_conversation_input_ids(tokenizer, query=query, history=[], images=[image], template_version='vqa')   # vqa mode
inputs = {
    'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
    'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
    'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
    'images': [[inputs['images'][0].to('cuda').to(torch.bfloat16)]],
}
gen_kwargs = {"max_length": 2048, "do_sample": False}

with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0]))

# 4</s>


ModuleNotFoundError: No module named 'xformers'